<a href="https://colab.research.google.com/github/leedhn/lab/blob/main/mnli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fairseq

In [ ]:
import torch
from fairseq.data.data_utils import collate_tokens

class Contradiction:
    def __init__(self):

        self.dir=''
        self.txt=[]
        self.batches=[] #Roberta에 넣을 batch 사이즈로 txt를 분할한 리스트
        self.result_list=[] # result들 txt파일별로 묶기 위해 만듦
        self.result=[] #Roberta prediction 결과 저장
        self.contradiction_list=[] #txt파일별로 cotradiction이 나온 pair 저장
        self.roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli') #사용할 모델
        self.roberta.cuda() #gpu사용
        self.roberta.eval()

    #txt파일 batch사이즈로 분할해서 self.batches에 저장
    def make_batch(self,size):
        self.batches = [self.txt[i*n:(i+1)*size] for i in range((len(self.txt)+size-1)//size)]
        #return self.batches

    # roberta 돌려서 contradiction 모으기 (메인 기능)
    def count_contradict(self):

        self.result_list=[]
        self.contradiction_list=[]
        #0=contradiction
        #1=natural
        #2=entailment

        k=0
        for batch_of_pairs in self.batches:
            
            self.batch = collate_tokens(
                [self.roberta.encode(pair[0], pair[1]) for pair in batch_of_pairs], pad_idx=1
            )

            logprobs = self.roberta.predict('mnli', self.batch)
            self.result=logprobs.argmax(dim=1)
            #print(k,'th batch')
            #print(result)
            self.result_list.append(self.result)
            for r in range(len(self.result)):
                if self.result[r]==0:
                    self.contradiction_list.append(batch_of_pairs[r])
                    
            k+=1
           
        #return (self.result_list, self.contradiction_list)

    # 디렉토리에서 파일 읽어서 self.txt에 저장
    def get_txt(self,dir): #txt 형식은 'ground truth' \t 'predict' \n ...
        self.txt=[]
        with open(dir,'r') as f:
            lines = f.readlines()
            for line in lines: 
                self.txt.append(line[:-1].split('\t'))
        #print(self.txt[:5][1])
        #return self.txt

    # 외부에서 클래스 함수 돌릴 때 사용. return값 있음
    def run(self,dir,size):
        self.get_txt(dir)
        print('run file dir:',dir)
        self.make_batch(size)
        self.count_contradict()
        len_contrad=len(self.contradiction_list)
        len_txt=len(self.txt)
        print(len_contrad)
        #print(self.result_list)
        print('file ',dir,'is done')
        return (len_contrad,len_txt,(len_contrad/len_txt), self.contradiction_list, self.result_list)

#사용 안 한 셀들

In [ ]:
txt=[]
import json
i=0
with open('/content/drive/MyDrive/persona/masked100_cont.txt','r') as f:
    lines = f.readlines()
    for line in lines: 
        i+=1
        txt.append(line[:-1].split('\t'))
print(txt)
print(i)

[['i am good , i just got off work and tired , i have two jobs .', 'yeah , well what about you ?'], ['i rather read , i have read about 20 books this year .', 'where do you work ?'], ['but a good movie is always good .', 'that is awesome . are you scared there ? i am so scared of clowns .'], ['i work in the movies as well .', 'i work in the movies as well .'], ['yes it is neat , i stunt double , it is so much fun and hard work .', 'yes it is neat , i stunt double , it is so much fun and hard work .'], ['nice , i only have one parent so now i help out my mom .', 'i do not know about that bro working from home does not pay too well .'], ['she raised me right , i am just like her .', 'she raised me right , i am just like her .'], ['now that i am older home depot is my toy r us .', 'now that i am older home depot is my toy r us .'], ['i am doing great . just relaxing with my two dogs .', 'i am mostly good . tired and some wicked heartburn . is that over sharing ? you ?'], ['that is neat . 

In [ ]:
len(txt)

7801

In [ ]:
n=16
batches = [txt[i*n:(i+1)*n] for i in range((len(txt)+n-1)//n)]

In [ ]:
import torch
from fairseq.data.data_utils import collate_tokens

roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.cuda()
roberta.eval()

result_list=[]
contradiction_list=[]
#0=contradiction
#1=natural
#2=entailment

k=1
c=0
for batch_of_pairs in batches:

    batch = collate_tokens(
        [roberta.encode(pair[0], pair[1]) for pair in batch_of_pairs], pad_idx=1
    )

    logprobs = roberta.predict('mnli', batch)
    result=logprobs.argmax(dim=1)
    print(k,'th batch')
    #print(result)
    result_list.append(result)
    for r in range(len(result)):
        if result[r]==0:
            contradiction_list.append(batch_of_pairs[r])
            c+=1
            #print(k,r, batch_of_pairs[r])
    k+=1
    
# tensor([0, 2, 1, 0])

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


1 th batch
2 th batch
3 th batch
4 th batch
5 th batch
6 th batch
7 th batch
8 th batch
9 th batch
10 th batch
11 th batch
12 th batch
13 th batch
14 th batch
15 th batch
16 th batch
17 th batch
18 th batch
19 th batch
20 th batch
21 th batch
22 th batch
23 th batch
24 th batch
25 th batch
26 th batch
27 th batch
28 th batch
29 th batch
30 th batch
31 th batch
32 th batch
33 th batch
34 th batch
35 th batch
36 th batch
37 th batch
38 th batch
39 th batch
40 th batch
41 th batch
42 th batch
43 th batch
44 th batch
45 th batch
46 th batch
47 th batch
48 th batch
49 th batch
50 th batch
51 th batch
52 th batch
53 th batch
54 th batch
55 th batch
56 th batch
57 th batch
58 th batch
59 th batch
60 th batch
61 th batch
62 th batch
63 th batch
64 th batch
65 th batch
66 th batch
67 th batch
68 th batch
69 th batch
70 th batch
71 th batch
72 th batch
73 th batch
74 th batch
75 th batch
76 th batch
77 th batch
78 th batch
79 th batch
80 th batch
81 th batch
82 th batch
83 th batch
84 th batch
8

In [ ]:
c

1124

In [ ]:
len(contradiction_list)

1124

In [ ]:
#사용 안 함
def make_dir_list(filename_list, file_dir,file_type):
    dir_list=[]
    for name in filename_list:
        s= file_dir+'/'+name+'.'+file_type
        dir_list.append(s)
    return(dir_list)

In [ ]:
#사용 안 함
filename_list=['masked20_cont','masked100_cont','biencoder_masked20_with_pl_cont','biencoder_masked100_with_pl_cont']

dir_list=make_dir_list(filename_list,'/content/drive/MyDrive/persona','txt')
dir_list

['/content/drive/MyDrive/persona/masked20_cont.txt',
 '/content/drive/MyDrive/persona/masked100_cont.txt',
 '/content/drive/MyDrive/persona/biencoder_masked20_with_pl_cont.txt',
 '/content/drive/MyDrive/persona/biencoder_masked100_with_pl_cont.txt']

# RUN!

In [ ]:
contrad_file={} #여기에 파일 별 결과들이 저장됩니다. key는 파일 이름. value는 결과값들.

batch_size=16 #추후에 수정 가능

#filename_list=['oracle_cont','masked100_cont']
filename_list=['masked20_cont','masked100_cont','biencoder_masked20_with_pl_cont','biencoder_masked100_with_pl_cont',
               'oracle_cont','ours_masked20_with_pl_cont','ours_masked100_with_pl_cont']

file_type='txt'
file_dir='/content/drive/MyDrive/persona'

model=Contradiction()
for name in filename_list:
    dir= file_dir+'/'+name+'.'+file_type
    print(dir)
    contrad_file[name]=model.run(dir,batch_size)
    # [contradiction 개수, txt 전체 개수, 비율, contradiction 전문, roberta prediction 리스트]

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


/content/drive/MyDrive/persona/masked20_cont.txt
run file dir: /content/drive/MyDrive/persona/masked20_cont.txt
566
file  /content/drive/MyDrive/persona/masked20_cont.txt is done
/content/drive/MyDrive/persona/masked100_cont.txt
run file dir: /content/drive/MyDrive/persona/masked100_cont.txt
1124
file  /content/drive/MyDrive/persona/masked100_cont.txt is done
/content/drive/MyDrive/persona/biencoder_masked20_with_pl_cont.txt
run file dir: /content/drive/MyDrive/persona/biencoder_masked20_with_pl_cont.txt
497
file  /content/drive/MyDrive/persona/biencoder_masked20_with_pl_cont.txt is done
/content/drive/MyDrive/persona/biencoder_masked100_with_pl_cont.txt
run file dir: /content/drive/MyDrive/persona/biencoder_masked100_with_pl_cont.txt
733
file  /content/drive/MyDrive/persona/biencoder_masked100_with_pl_cont.txt is done
/content/drive/MyDrive/persona/oracle_cont.txt
run file dir: /content/drive/MyDrive/persona/oracle_cont.txt
420
file  /content/drive/MyDrive/persona/oracle_cont.txt is d

In [ ]:
#결과 출력
for key in contrad_file.keys():

    print((key+".txt").ljust(40),('contrad count: '+str(contrad_file[key][0])).center(20),
          ('txt count: '+str(contrad_file[key][1])).center(20),
          ('contradict ratio: '+ str(round(contrad_file[key][2],3))).rjust(40),sep='')

masked20_cont.txt                        contrad count: 566   txt count: 7801                    contradict ratio: 0.073
masked100_cont.txt                      contrad count: 1124   txt count: 7801                    contradict ratio: 0.144
biencoder_masked20_with_pl_cont.txt      contrad count: 497   txt count: 7801                    contradict ratio: 0.064
biencoder_masked100_with_pl_cont.txt     contrad count: 733   txt count: 7801                    contradict ratio: 0.094
oracle_cont.txt                          contrad count: 420   txt count: 7801                    contradict ratio: 0.054
ours_masked20_with_pl_cont.txt           contrad count: 453   txt count: 7801                    contradict ratio: 0.058
ours_masked100_with_pl_cont.txt          contrad count: 610   txt count: 7801                    contradict ratio: 0.078
